In [15]:
from __future__ import print_function
from vowpalwabbit import pyvw

In [2]:
tag = {}
tag['CC'] = 1
tag['CD'] = 2
tag['DT'] = 3
tag['EX'] = 3
tag['FW'] = 4
tag['IN'] = 5
tag['JJ'] = 6
tag['JJR'] = 6
tag['JJS'] = 6
tag['MD'] = 7
tag['NN'] = 8
tag['NNP'] = 8
tag['NNPS'] = 8
tag['NNS'] = 8
tag['PDT'] = 3
tag['POS'] = 11
tag['PRP'] = 9
tag['PRP$'] = 9
tag['RB'] = 10
tag['RBR'] = 10
tag['RBS'] = 10
tag['RP'] = 11
tag['SYM'] = 12
tag['TO'] = 11
tag['UH'] = 13
tag['VB'] = 14
tag['VBD'] = 14
tag['VBG'] = 14
tag['VBN'] = 14
tag['VBP'] = 14
tag['VBZ'] = 14
tag['WDT'] = 3
tag['WP'] = 9
tag['WP$'] = 9
tag['WRB'] = 10

In [3]:
s = ''
punc = ['#', '$',"''",'(',')',',','.',':',"``"]
sentence = []
sentences = []
pos_tags = set([])
file = open('train.txt', 'r') 
for line in file:
    if line == '\n': 
        sentences.append(sentence)
        sentence = []
        continue
    parts = line.split()
    if parts[1] in punc:
        continue
    pos_tags.add(parts[1])
    tup = (tag[parts[1]],parts[0])
    sentence.append(tup)

In [4]:
print (sentences[0][0][0])
print (sentences[0][0][1])

8
Confidence


In [21]:
for aSent in sentences[:1000]:
    for aTuple in aSent:
        pos,word = aTuple
        if word.lower().startswith("pre"):
            print (word)
            print (pos)

prevent
14
pressure
8
prepared
14
preoccupied
14
predict
14
previously
10
pressure
8
president
8
present
6
previous
6
previous
6
previous
6
president
8
president
8
pressure
14
president
8
pretext
8
present
14
presidential
6
president
8
president
8
previous-month
6
president
8
previous
6
previous
6
president
8
president
8
previous
6
pretax
6
pretax
6
prefer
14
president
8
pressure
8
pressure
8
predicted
14
president
8
president
8
president
8
president
8
president
8
president
8
previously
10
president
8
president
8
president
8
president
8
president
8
president
8
presidents
8
preparing
14
pressing
14
pressure
8
previously
10
press
14
pressure
8
prepay
14
president
8
president
8
president
8
president
8
presidency
8
prepared
14
president
8
president
8
president
8
pressed
14
prepared
14
pressure
8
president
8
presented
14
presentations
8
presented
14
previously
10
previously
10
president
8
president
8
precipitated
14
present
6
preferring
14
precious
6
precious
6
precious
6
president
8
premiu

In [6]:
sentence_test = []
sentences_test = []
pos_tags_test = set([])
file = open('test.txt', 'r') 
for line in file:
    if line == '\n': 
        sentences_test.append(sentence_test)
        sentence_test = []
        continue
    parts = line.split()
    if parts[1] in punc:
        continue
    pos_tags_test.add(parts[1])
    tup = (tag[parts[1]],parts[0])
    sentence_test.append(tup)

In [ ]:
class SequenceLabeler(pyvw.SearchTask):
    def __init__(self, vw, sch, num_actions):
        # you must must must initialize the parent class
        # this will automatically store self.sch <- sch, self.vw <- vw
        pyvw.SearchTask.__init__(self, vw, sch, num_actions)
        
        # set whatever options you want
        sch.set_options( sch.AUTO_HAMMING_LOSS | sch.AUTO_CONDITION_FEATURES )

    def _run(self, sentence):   # it's called _run to remind you that you shouldn't call it directly!
        output = []
        for n in range(len(sentence)):
            pos,word = sentence[n]
            # use "with...as..." to guarantee that the example is finished properly
            with self.vw.example({'w': [word]}) as ex:
                pred = self.sch.predict(examples=ex, my_tag=n+1, oracle=pos, condition=[(n,'p'), (n-1, 'q')])
                output.append(pred)
        return output

## Customizing features


In [40]:
class SequenceLabeler2(pyvw.SearchTask):
    def __init__(self, vw, sch, num_actions):
        pyvw.SearchTask.__init__(self, vw, sch, num_actions)
        sch.set_options(sch.AUTO_HAMMING_LOSS)

    def _run(self, sentence):
        output = []
        #loss = 0.
        for n in range(len(sentence)):
            pos,word = sentence[n]
            prevPred = output[n-1] if n > 0 else '<s>'
            prev_n_2 = output[n-2] if n > 1 else '<s>'
            posnew, prevWord = sentence[n-1] if n> 0 else ('<s>','<s>')
            with self.vw.example({'w': [word],'p': [prevPred], 'r':[prevWord]}) as ex: 
                pred = self.sch.predict(examples=ex, my_tag=n+1, oracle=pos, condition=[(n,'p'),(n,'r')])
                output.append(pred)
                
        return output
    
#sequenceLabeler2 = vw.init_search_task(SequenceLabeler2)
#sequenceLabeler2.learn(my_dataset)
#print(sequenceLabeler2.predict( [(0,w) for w in "the sandwich ate a monster".split()] ))

## Customizing loss functions

In [125]:
class SequenceLabeler3(pyvw.SearchTask):
    def __init__(self, vw, sch, num_actions):
        pyvw.SearchTask.__init__(self, vw, sch, num_actions)
        sch.set_options(sch.AUTO_CONDITION_FEATURES)

    def _run(self, sentence):
        output = []
        loss = 0.
        for n in range(len(sentence)):
            pos,word = sentence[n]
            with self.vw.example({'w': [word]}) as ex: 
                pred = self.sch.predict(examples=ex, my_tag=n+1, oracle=pos, condition=[(n,'p'),(n-1,'q')])
                output.append(pred)
                if pred != pos:
                    loss += wt_err[pos]            
        self.sch.loss(loss)
                
        return output

In [6]:
class SequenceLabeler4(pyvw.SearchTask):
    def __init__(self, vw, sch, num_actions):
        pyvw.SearchTask.__init__(self, vw, sch, num_actions)
        sch.set_options(sch.AUTO_CONDITION_FEATURES)

    def _run(self, sentence):
        output = []
        loss = 0.
        for n in range(len(sentence)):
            pos,word = sentence[n]
            with self.vw.example({'w': [word]}) as ex: 
                pred = self.sch.predict(examples=ex, my_tag=n+1, oracle=pos, condition=[(n,'p'),(n-1,'q')])
                output.append(pred)
                if pred != pos:
                    loss += wt[pos]            
        self.sch.loss(loss)
                
        return output

In [6]:
class SequenceLabeler5(pyvw.SearchTask):
    def __init__(self, vw, sch, num_actions):
        pyvw.SearchTask.__init__(self, vw, sch, num_actions)
        sch.set_options(sch.AUTO_CONDITION_FEATURES)

    def _run(self, sentence):
        output = []
        loss = 0.
        for n in range(len(sentence)):
            pos,word = sentence[n]
            with self.vw.example({'w': [word]}) as ex: 
                pred = self.sch.predict(examples=ex, my_tag=n+1, oracle=pos, condition=[(n,'p'),(n-1,'q')])
                output.append(pred)
                if pred != pos:
                    loss = 1           
        self.sch.loss(loss)
                
        return output

### Ratnaparkhi 1

In [7]:
class SequenceLabelerRp1(pyvw.SearchTask):
    def __init__(self, vw, sch, num_actions):
        pyvw.SearchTask.__init__(self, vw, sch, num_actions)
        #sch.set_options(sch.AUTO_CONDITION_FEATURES)
        sch.set_options(sch.AUTO_HAMMING_LOSS)

    def _run(self, sentence):
        output = []
        loss = 0.
        for n in range(len(sentence)):
            pos,word = sentence[n]
            prevPred = output[n-1] if n > 0 else '<s>'
            #prev_n_2 = output[n-2] if n > 1 else '<s>'
            #posnew, prevWord = sentence[n-1] if n> 0 else ('<s>','<s>')
            #with self.vw.example({'w': [word],'p': [prevPred], 'r':[prevWord]}) as ex:
            
            # ing feature
            f1 = 0
            #if word.endswith("ing") and (pos == 14):
            if word.lower().endswith("ing"):
                f1 = 1
            
            # pre feature
            f2 = 0
            #if word.startswith("pre") and (pos == 8):
            if word.lower().startswith("pre"):
                f2 = 1
            
            with self.vw.example({'w': [word], 'p': [prevPred], 'q': [f1], 'r': [f2]}) as ex:
                pred = self.sch.predict(examples=ex, my_tag=n+1, oracle=pos, condition=[(n,'p'),(n,'q'),(n,'r')])
                output.append(pred)
                if pred != pos:
                    loss = 1           
        #self.sch.loss(loss)
                
        return output
    
#sequenceLabeler2 = vw.init_search_task(SequenceLabeler2)
#sequenceLabeler2.learn(my_dataset)
#print(sequenceLabeler2.predict( [(0,w) for w in "the sandwich ate a monster".split()] ))

### Ratnaparkhi 2

In [7]:
class SequenceLabelerRp2(pyvw.SearchTask):
    def __init__(self, vw, sch, num_actions):
        pyvw.SearchTask.__init__(self, vw, sch, num_actions)
        #sch.set_options(sch.AUTO_CONDITION_FEATURES)
        sch.set_options(sch.AUTO_HAMMING_LOSS)

    def _run(self, sentence):
        output = []
        loss = 0.
        for n in range(len(sentence)):
            pos,word = sentence[n]
            prevPred = output[n-1] if n > 0 else '<s>'
            #prev_n_2 = output[n-2] if n > 1 else '<s>'
            posprev, prevWord = sentence[n-1] if n> 0 else ('<s>','<s>')
            posnext, nextWord = sentence[n+1] if (n+1)< len(sentence) else ('<s>','<s>')
            #with self.vw.example({'w': [word],'p': [prevPred], 'r':[prevWord]}) as ex:
            
            # ing feature
            f1 = 0
            #if word.endswith("ing") and (pos == 14):
            if word.lower().endswith("ing"):
                f1 = 1
            
            # pre feature
            f2 = 0
            #if word.startswith("pre") and (pos == 8):
            if word.lower().startswith("pre"):
                f2 = 1
            
            # the @ n-1 feature
            f3 = 0
            #if word.startswith("pre") and (pos == 8):
            if prevWord.lower() == "the":
                f3 = 1
            
            # the @ n+1 feature
            f4 = 0
            #if word.startswith("pre") and (pos == 8):
            if nextWord.lower() == "the":
                f4 = 1
            
            with self.vw.example({'w': [word], 'p': [prevPred], 'q': [f3], 'r': [f4]}) as ex:
                pred = self.sch.predict(examples=ex, my_tag=n+1, oracle=pos, condition=[(n,'p'),(n,'q'),(n,'r')])
                output.append(pred)
                if pred != pos:
                    loss = 1           
        #self.sch.loss(loss)
                
        return output
    
#sequenceLabeler2 = vw.init_search_task(SequenceLabeler2)
#sequenceLabeler2.learn(my_dataset)
#print(sequenceLabeler2.predict( [(0,w) for w in "the sandwich ate a monster".split()] ))

### Gold

In [6]:
class SequenceLabelerGold(pyvw.SearchTask):
    def __init__(self, vw, sch, num_actions):
        pyvw.SearchTask.__init__(self, vw, sch, num_actions)
        #sch.set_options(sch.AUTO_CONDITION_FEATURES)
        #sch.set_options(sch.AUTO_HAMMING_LOSS)

    def _run(self, sentence):
        output = []
        loss = 0.
        for n in range(len(sentence)):
            pos,word = sentence[n]
            prevPred = output[n-1] if n > 0 else '<s>'
            prev_n_2 = output[n-2] if n > 1 else '<s>'
            posnew, prevWord = sentence[n-1] if n> 0 else ('<s>','<s>')
            with self.vw.example({'w': [word],'p': [prevPred], 'r':[prevWord]}) as ex: 
                pred = self.sch.predict(examples=ex, my_tag=n+1, oracle=pos, condition=[(n,'p'),(n,'r')])
                output.append(pred)
                if pred != pos:
                    loss = 1           
        self.sch.loss(loss)
                
        return output
    
#sequenceLabeler2 = vw.init_search_task(SequenceLabeler2)
#sequenceLabeler2.learn(my_dataset)
#print(sequenceLabeler2.predict( [(0,w) for w in "the sandwich ate a monster".split()] ))

In [8]:
def train_loss():
    print("Calculating training loss...")
    global tag_count
    global tag_error
    tag_count = {}
    tag_error = {}
    loss_t=[]
    s=0
    for sen in sentences[:1000]:
        s+=1
        temp=[]
        for w in sen:
            temp.append((1,w[1]))
        out = sequenceLabeler.predict(temp)
        #print(out)
        loss=0
        c=0
        for i in out:        
            if sen[c][0] in tag_count:
                tag_count[sen[c][0]] += 1
            else:
                tag_count[sen[c][0]] = float(1)
                tag_error[sen[c][0]] = float(0)

            if i != sen[c][0]:
                loss+=1
                tag_error[sen[c][0]] += 1
                #print ("Predicted value at ",c," is ",i)
                #print (sentences[0][c])
            c+=1
        loss_t.append(loss/float(c))

    print ("Training Loss:")
    print (sum(loss_t) / float(len(loss_t)))

In [9]:
def test_loss():
    print("Calculating test loss...")
    global tag_count
    global tag_error
    tag_count = {}
    tag_error = {}
    loss_t=[]
    s=0
    for sen in sentences_test:
        s+=1
        temp=[]
        for w in sen:
            temp.append((1,w[1]))
        out = sequenceLabeler.predict(temp)
        #print(out)
        loss=0
        c=0
        for i in out:        
            if sen[c][0] in tag_count:
                tag_count[sen[c][0]] += 1
            else:
                tag_count[sen[c][0]] = float(1)
                tag_error[sen[c][0]] = float(0)

            if i != sen[c][0]:
                loss+=1
                tag_error[sen[c][0]] += 1
                #print ("Predicted value at ",c," is ",i)
                #print (sentences[0][c])
            c+=1
        loss_t.append(loss/float(c))

    print ("Test Loss:")
    print (sum(loss_t) / float(len(loss_t)))

In [10]:
import operator
def updateWeights():
    print("Updating weights...")
    tag_loss = {}
    sum_err=0
    for t in tag_count:
        tag_loss[t] = tag_error[t]/tag_count[t]
        sum_err+=tag_error[t]
        
    sorted_loss = sorted(tag_loss.items(), key=operator.itemgetter(1))

    for tp in sorted_loss:
            print( tp[0], " = ", tag_error[tp[0]], "/", tag_count[tp[0]], " = ", tp[1] )
            wt_err[tp[0]]=tp[1]*(tag_error[tp[0]]/sum_err)
            
    avg_err=sum(wt_err)/14.0
    wt_err[:]=[1+(x - avg_err) for x in wt_err]        

In [11]:
vw = pyvw.vw("--search 14 --audit --quiet --search_task hook --ring_size 1024 -f pos_tagger.model")

In [12]:
sequenceLabeler = vw.init_search_task(SequenceLabelerRp2)

In [13]:
wt_err=[1]*15
tag_count = {}
tag_error = {}

### Vanilla

In [23]:
for i in range(5):
    print (i)
    sequenceLabeler.learn(sentences[:1000])
    tag_count = {}
    tag_error = {}
    train_loss()
    test_loss()
    #updateWeights()

0
Calculating training loss...
Training Loss:
0.0472563791476
Calculating loss...
1000
2000
Test loss
0.136943661398
1
Calculating training loss...
Training Loss:
0.0442743932739
Calculating loss...
1000
2000
Test loss
0.135636213701
2
Calculating training loss...
Training Loss:
0.0448799255814
Calculating loss...
1000
2000
Test loss
0.136534322135
3
Calculating training loss...
Training Loss:
0.0447805006025
Calculating loss...
1000
2000
Test loss
0.136985796514
4
Calculating training loss...
Training Loss:
0.0449880803822
Calculating loss...
1000
2000
Test loss
0.137206436963


In [30]:
train_loss()
updateWeights()
test_loss()
updateWeights()

Calculating training loss...
Training Loss:
0.0449880803822
Updating weights...
1  =  0.0 / 1150.0  =  0.0
4  =  0.0 / 14.0  =  0.0
7  =  0.0 / 470.0  =  0.0
5  =  30.0 / 5174.0  =  0.00579822187862
11  =  12.0 / 1612.0  =  0.00744416873449
9  =  16.0 / 1170.0  =  0.0136752136752
3  =  156.0 / 4380.0  =  0.0356164383562
8  =  602.0 / 14766.0  =  0.0407693349587
6  =  242.0 / 3430.0  =  0.0705539358601
2  =  124.0 / 1742.0  =  0.0711825487945
14  =  488.0 / 6340.0  =  0.0769716088328
10  =  160.0 / 1566.0  =  0.102171136654
Calculating test loss...
Test Loss:
0.137206436963
Updating weights...
11  =  12.0 / 1624.0  =  0.00738916256158
1  =  11.0 / 1214.0  =  0.00906095551895
5  =  70.0 / 5071.0  =  0.0138039834352
7  =  7.0 / 470.0  =  0.0148936170213
9  =  30.0 / 1349.0  =  0.0222386953299
3  =  172.0 / 4280.0  =  0.0401869158879
8  =  1470.0 / 14612.0  =  0.10060224473
14  =  1446.0 / 6232.0  =  0.232028241335
4  =  1.0 / 4.0  =  0.25
10  =  422.0 / 1567.0  =  0.269304403318
6  =  116

### Static weight updates

In [13]:
wt = {}

wt[1] = 1.0 
wt[2] = 1.0
wt[3] = 1.0
wt[4] = 1.0
wt[5] = 1.0
wt[6] = 1.2
wt[7] = 1.0
wt[8] = 1.1
wt[9] = 1.0
wt[10] = 1.2
wt[11] = 1.0
wt[12] = 1.0
wt[13] = 1.0
wt[14] = 1.1

In [14]:
for i in range(5):
    print (i)
    sequenceLabeler.learn(sentences[:1000])
    tag_count = {}
    tag_error = {}
    train_loss()
    test_loss()
    updateWeights()

0
Calculating training loss...
Training Loss:
0.0509687903932
Calculating test loss...
Test Loss:
0.137398884444
Updating weights...
11  =  16.0 / 1624.0  =  0.00985221674877
1  =  13.0 / 1214.0  =  0.0107084019769
7  =  8.0 / 470.0  =  0.0170212765957
5  =  88.0 / 5071.0  =  0.0173535791757
3  =  174.0 / 4280.0  =  0.0406542056075
9  =  60.0 / 1349.0  =  0.0444773906597
8  =  1239.0 / 14612.0  =  0.0847933205584
14  =  1451.0 / 6232.0  =  0.23283055199
10  =  434.0 / 1567.0  =  0.276962348437
6  =  1190.0 / 3243.0  =  0.366944187481
2  =  789.0 / 1918.0  =  0.411366006257
4  =  3.0 / 4.0  =  0.75
13  =  2.0 / 2.0  =  1.0
1
Calculating training loss...
Training Loss:
0.0447766393709
Calculating test loss...
Test Loss:
0.135841969592
Updating weights...
11  =  12.0 / 1624.0  =  0.00738916256158
1  =  11.0 / 1214.0  =  0.00906095551895
5  =  72.0 / 5071.0  =  0.0141983829619
7  =  8.0 / 470.0  =  0.0170212765957
9  =  32.0 / 1349.0  =  0.0237212750185
3  =  172.0 / 4280.0  =  0.040186915

### Static weight updates 2.0

In [13]:
wt = {}

wt[1] = 1.0 
wt[2] = 1.0
wt[3] = 1.0
wt[4] = 1.0
wt[5] = 1.0
wt[6] = 1.4
wt[7] = 1.0
wt[8] = 1.25
wt[9] = 1.0
wt[10] = 1.4
wt[11] = 1.0
wt[12] = 1.0
wt[13] = 1.0
wt[14] = 1.25

In [14]:
for i in range(5):
    print (i)
    sequenceLabeler.learn(sentences[:1000])
    tag_count = {}
    tag_error = {}
    train_loss()
    test_loss()
    updateWeights()

0
Calculating training loss...
Training Loss:
0.0562226198542
Calculating test loss...
Test Loss:
0.138625882943
Updating weights...
11  =  16.0 / 1624.0  =  0.00985221674877
1  =  13.0 / 1214.0  =  0.0107084019769
5  =  99.0 / 5071.0  =  0.0195227765727
7  =  18.0 / 470.0  =  0.0382978723404
3  =  177.0 / 4280.0  =  0.0413551401869
9  =  66.0 / 1349.0  =  0.0489251297257
8  =  1189.0 / 14612.0  =  0.0813714754996
14  =  1462.0 / 6232.0  =  0.23459563543
10  =  444.0 / 1567.0  =  0.283343969368
6  =  1204.0 / 3243.0  =  0.371261177922
2  =  832.0 / 1918.0  =  0.433785192909
4  =  3.0 / 4.0  =  0.75
13  =  2.0 / 2.0  =  1.0
1
Calculating training loss...
Training Loss:
0.0451226472155
Calculating test loss...
Test Loss:
0.135988618917
Updating weights...
1  =  11.0 / 1214.0  =  0.00906095551895
11  =  16.0 / 1624.0  =  0.00985221674877
5  =  76.0 / 5071.0  =  0.0149871820154
7  =  8.0 / 470.0  =  0.0170212765957
9  =  35.0 / 1349.0  =  0.0259451445515
3  =  174.0 / 4280.0  =  0.04065420

### 0/1 loss

In [13]:
for i in range(5):
    print (i)
    sequenceLabeler.learn(sentences[:1000])
    tag_count = {}
    tag_error = {}
    train_loss()
    test_loss()
    updateWeights()

0
Calculating training loss...
Training Loss:
0.0479161243961
Calculating test loss...
Test Loss:
0.136344372717
Updating weights...
1  =  11.0 / 1214.0  =  0.00906095551895
11  =  21.0 / 1624.0  =  0.0129310344828
5  =  94.0 / 5071.0  =  0.0185367777559
7  =  9.0 / 470.0  =  0.0191489361702
9  =  46.0 / 1349.0  =  0.0340993328391
3  =  163.0 / 4280.0  =  0.0380841121495
8  =  1377.0 / 14612.0  =  0.0942376129209
14  =  1299.0 / 6232.0  =  0.208440308087
10  =  444.0 / 1567.0  =  0.283343969368
6  =  1242.0 / 3243.0  =  0.382978723404
2  =  755.0 / 1918.0  =  0.393639207508
4  =  3.0 / 4.0  =  0.75
13  =  2.0 / 2.0  =  1.0
1
Calculating training loss...
Training Loss:
0.0411543364836
Calculating test loss...
Test Loss:
0.134548744193
Updating weights...
1  =  11.0 / 1214.0  =  0.00906095551895
11  =  18.0 / 1624.0  =  0.0110837438424
5  =  82.0 / 5071.0  =  0.0161703805955
7  =  8.0 / 470.0  =  0.0170212765957
9  =  31.0 / 1349.0  =  0.0229799851742
3  =  157.0 / 4280.0  =  0.036682242

### Gold

In [13]:
for i in range(5):
    print (i)
    sequenceLabeler.learn(sentences[:5000])
    tag_count = {}
    tag_error = {}
    train_loss()
    test_loss()
    updateWeights()

0
Calculating training loss...
Training Loss:
0.0673703156967
Calculating test loss...
Test Loss:
0.102423956541
Updating weights...
11  =  12.0 / 1624.0  =  0.00738916256158
1  =  12.0 / 1214.0  =  0.00988467874794
7  =  7.0 / 470.0  =  0.0148936170213
5  =  90.0 / 5071.0  =  0.0177479787024
9  =  26.0 / 1349.0  =  0.0192735359526
3  =  115.0 / 4280.0  =  0.0268691588785
8  =  953.0 / 14612.0  =  0.0652203668218
14  =  904.0 / 6232.0  =  0.145057766367
10  =  382.0 / 1567.0  =  0.243777919592
2  =  586.0 / 1918.0  =  0.305526590198
6  =  1068.0 / 3243.0  =  0.329324699352
4  =  3.0 / 4.0  =  0.75
13  =  2.0 / 2.0  =  1.0
1
Calculating training loss...
Training Loss:
0.0494833066321
Calculating test loss...
Test Loss:
0.0986787942501
Updating weights...
11  =  12.0 / 1624.0  =  0.00738916256158
1  =  10.0 / 1214.0  =  0.00823723228995
7  =  7.0 / 470.0  =  0.0148936170213
9  =  23.0 / 1349.0  =  0.0170496664196
5  =  96.0 / 5071.0  =  0.0189311772826
3  =  95.0 / 4280.0  =  0.022196261

### Ratnaparkhi oracle aided

In [25]:
for i in range(5):
    print (i)
    sequenceLabeler.learn(sentences[:1000])
    tag_count = {}
    tag_error = {}
    train_loss()
    test_loss()
    updateWeights()

0
Calculating training loss...
Training Loss:
0.0500827863288
Calculating test loss...
Test Loss:
0.139858832411
Updating weights...
1  =  11.0 / 1214.0  =  0.00906095551895
11  =  22.0 / 1624.0  =  0.0135467980296
7  =  8.0 / 470.0  =  0.0170212765957
5  =  97.0 / 5071.0  =  0.0191283770459
9  =  49.0 / 1349.0  =  0.0363232023721
3  =  162.0 / 4280.0  =  0.0378504672897
8  =  1495.0 / 14612.0  =  0.10231316726
14  =  1367.0 / 6232.0  =  0.219351732991
10  =  436.0 / 1567.0  =  0.278238672623
6  =  1231.0 / 3243.0  =  0.379586802344
2  =  756.0 / 1918.0  =  0.394160583942
4  =  4.0 / 4.0  =  1.0
13  =  2.0 / 2.0  =  1.0
1
Calculating training loss...
Training Loss:
0.0426078653255
Calculating test loss...
Test Loss:
0.134364919634
Updating weights...
1  =  11.0 / 1214.0  =  0.00906095551895
11  =  18.0 / 1624.0  =  0.0110837438424
7  =  7.0 / 470.0  =  0.0148936170213
5  =  77.0 / 5071.0  =  0.0151843817787
9  =  30.0 / 1349.0  =  0.0222386953299
3  =  157.0 / 4280.0  =  0.036682242990

### Ratnaparkhi 1 w/o oracle

In [14]:
for i in range(5):
    print (i)
    sequenceLabeler.learn(sentences[:1000])
    tag_count = {}
    tag_error = {}
    train_loss()
    test_loss()
    updateWeights()

0
Calculating training loss...
Training Loss:
0.0482208012741
Calculating test loss...
Test Loss:
0.132987339159
Updating weights...
1  =  11.0 / 1214.0  =  0.00906095551895
11  =  22.0 / 1624.0  =  0.0135467980296
7  =  8.0 / 470.0  =  0.0170212765957
5  =  97.0 / 5071.0  =  0.0191283770459
9  =  46.0 / 1349.0  =  0.0340993328391
3  =  164.0 / 4280.0  =  0.0383177570093
8  =  1514.0 / 14612.0  =  0.103613468382
14  =  1026.0 / 6232.0  =  0.164634146341
10  =  437.0 / 1567.0  =  0.278876834716
6  =  1243.0 / 3243.0  =  0.383287079864
2  =  751.0 / 1918.0  =  0.391553701773
4  =  3.0 / 4.0  =  0.75
13  =  2.0 / 2.0  =  1.0
1
Calculating training loss...
Training Loss:
0.041023989362
Calculating test loss...
Test Loss:
0.130220510074
Updating weights...
1  =  11.0 / 1214.0  =  0.00906095551895
11  =  18.0 / 1624.0  =  0.0110837438424
5  =  77.0 / 5071.0  =  0.0151843817787
7  =  8.0 / 470.0  =  0.0170212765957
9  =  28.0 / 1349.0  =  0.0207561156412
3  =  157.0 / 4280.0  =  0.03668224299

### Ratnaparkhi 2

In [14]:
for i in range(5):
    print (i)
    sequenceLabeler.learn(sentences[:1000])
    tag_count = {}
    tag_error = {}
    train_loss()
    test_loss()
    updateWeights()

0
Calculating training loss...
Training Loss:
0.0482208012741
Calculating test loss...
Test Loss:
0.132987339159
Updating weights...
1  =  11.0 / 1214.0  =  0.00906095551895
11  =  22.0 / 1624.0  =  0.0135467980296
7  =  8.0 / 470.0  =  0.0170212765957
5  =  97.0 / 5071.0  =  0.0191283770459
9  =  46.0 / 1349.0  =  0.0340993328391
3  =  164.0 / 4280.0  =  0.0383177570093
8  =  1514.0 / 14612.0  =  0.103613468382
14  =  1026.0 / 6232.0  =  0.164634146341
10  =  437.0 / 1567.0  =  0.278876834716
6  =  1243.0 / 3243.0  =  0.383287079864
2  =  751.0 / 1918.0  =  0.391553701773
4  =  3.0 / 4.0  =  0.75
13  =  2.0 / 2.0  =  1.0
1
Calculating training loss...
Training Loss:
0.041023989362
Calculating test loss...
Test Loss:
0.130220510074
Updating weights...
1  =  11.0 / 1214.0  =  0.00906095551895
11  =  18.0 / 1624.0  =  0.0110837438424
5  =  77.0 / 5071.0  =  0.0151843817787
7  =  8.0 / 470.0  =  0.0170212765957
9  =  28.0 / 1349.0  =  0.0207561156412
3  =  157.0 / 4280.0  =  0.03668224299